In [1]:
## import 套件
import torch
import pandas as pd
import numpy as np
import torch.nn as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import AdamW ##新ㄉ 好像比較好
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import BertModel
from transformers import BertConfig

from matplotlib import pyplot as plt

In [4]:
## load data & path
columns_name = ['type','title','text','time','url']
dfchina = pd.read_csv('./data_after_sep/cn_all.tsv',sep = '\t',names = columns_name) ##中時
dffree = pd.read_csv('./data_after_sep/ltn_all.tsv',sep = '\t',names = columns_name) ##自由
dfunite = pd.read_csv('./data_after_sep/udn_all.tsv',sep = '\t',names = columns_name) ##聯合

model_path = './bert_pretrain_news/'
# model_path = './chinese_wwm_pytorch/'
tokenizer = BertTokenizer.from_pretrained(model_path)

In [5]:
##幫它們標label
for i in range(dfchina.shape[0]):
    dfchina.iloc[i][0] = 0
for i in range(dffree.shape[0]):
    dffree.iloc[i][0] = 1
for i in range(dfunite.shape[0]):
    dfunite.iloc[i][0] = 2
train_china = dfchina['text'].tolist()
train_free = dffree['text'].tolist()
train_unite = dfunite['text'].tolist()

china_y = dfchina['type'].tolist()
free_y = dffree['type'].tolist()
unite_y = dfunite['type'].tolist()

In [6]:
print(len(train_china))
print(len(train_free))
print(len(train_unite))

24070
32961
38713


In [7]:
##過濾一堆 nan
train_china = [data for data in train_china if type(data) != float]
train_free = [data for data in train_free if type(data) != float]
train_unite = [data for data in train_unite if type(data) != float] 
print(len(train_china))
print(len(train_free))
print(len(train_unite))

24069
32599
38612


In [8]:
china_y = [china_y[count] for count in range(len(train_china))]
free_y = [free_y[count] for count in range(len(train_free))]
unite_y = [unite_y[count] for count in range(len(train_unite))]
print(len(china_y))
print(len(free_y))
print(len(unite_y))

24069
32599
38612


In [9]:
## 切割train跟 test test取比例篇 先多一點
train_x = train_china[5000:]+train_free[6800:]+train_unite[8000:]
print(len(train_x))
train_y = china_y[5000:]+free_y[6800:]+unite_y[8000:]
print(len(train_y))
test_x = train_china[:5000]+train_free[:6800]+train_unite[:8000]
print(len(test_x))
test_y = china_y[:5000]+free_y[:6800]+unite_y[:8000]
print(len(test_y))

75480
75480
19800
19800


In [10]:
class TrainDataset(Dataset):
    def __init__(self, input_dict, y):
        self.input_ids = input_dict['input_ids']
        self.token_type_ids = input_dict['token_type_ids']
        self.attention_mask = input_dict['attention_mask']
        self.y = y
    def __getitem__(self, idx):
        input_id = self.input_ids[idx]
        tokentype = self.token_type_ids[idx]
        attentionmask = self.attention_mask[idx]
        y = self.y[idx]
        return input_id, tokentype, attentionmask, y
    def __len__(self):
        return len(self.input_ids)

In [16]:
train_input_dict = tokenizer.batch_encode_plus(train_x,
                                              add_special_tokens = True,
                                              max_length = 512,
                                              truncation = True,                ##是否截斷
                                              return_special_tokens_mask = True,
                                              pad_to_max_length = True,
                                              return_tensors = 'pt')
TRAIN_BATCH_SIZE = 4
train_y = np.array(train_y)       ##np.array
trainset = TrainDataset(train_input_dict, train_y) ##trainset參數如init
trainloader = DataLoader(trainset, batch_size = TRAIN_BATCH_SIZE, shuffle = True)  

In [11]:
test_input_dict = tokenizer.batch_encode_plus(test_x,
                                              add_special_tokens = True,
                                              max_length = 512,
                                              truncation = True,                ##是否截斷
                                              return_special_tokens_mask = True,
                                              pad_to_max_length = True,
                                              return_tensors = 'pt')
BATCH_SIZE = 64
test_y = np.array(test_y)       ##np.array
testset = TrainDataset(test_input_dict, test_y) ##trainset參數如init
testloader = DataLoader(testset, batch_size = BATCH_SIZE, shuffle = True)  

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [12]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
## 類別對應 0：政治 1：生活 2：國際 3：體育 4：娛樂 5：社會 6：財經
def get_test_acc(model, testloader):
    model.eval()  ##test mode
    total = 0 ##total_num
    correct = 0 ##correct_num
    with torch.no_grad():   ##eval不計算gradient 
        for data in testloader:
            tokens_tensors , segment_tensors,masks_tensors,labels = [t.to(device) for t in data]
            outputs = model(input_ids = tokens_tensors,
                           token_type_ids = segment_tensors,
                           attention_mask = masks_tensors,
                           labels = labels)
            pred = torch.argmax(outputs[1],dim=-1)
            total += labels.size()[0]
            correct += (pred == labels).sum().item()
    
    return correct/total

In [17]:
num_labels = 3
model = BertForSequenceClassification.from_pretrained(model_path,num_labels = num_labels)
model = model.to(device)
model.load_state_dict(torch.load('./BERT_news_2e5_0.pkl'),strict = False)
model.eval()

test_acc = get_test_acc(model,testloader)
print('test acc:' , test_acc)

Some weights of the model checkpoint at ./bert_pretrain_news/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not init

test acc: 0.6435858585858586


In [ ]:
## BertForSequencelassification
num_labels = 3
model = BertForSequenceClassification.from_pretrained(model_path,num_labels = num_labels)
model = model.to(device)
model.train()


EPOCHS = 5

optimizer = AdamW(model.parameters(),lr = 2e-5)

for epoch in range(0,EPOCHS):
    i = 0
    running_loss = 0.0
    correct = 0
    total = 0

    for (i,data) in enumerate(trainloader):
        
        tokens_tensors ,  segments_tensors , masks_tensors , labels  = [t.to(device) for t in data]
#         print(data)
        bert_outputs = model(input_ids=tokens_tensors, 
                             token_type_ids=segments_tensors, 
                             attention_mask=masks_tensors,
                             labels = labels)
        
        optimizer.zero_grad()
        logits = bert_outputs[1]
        ##權重
        pred = torch.argmax(logits,dim = -1)
        loss = bert_outputs[0] 
        loss.backward()
        optimizer.step()
        
        total += pred.size()[0]
        correct += (pred == labels).sum().item()
        running_loss += loss.item()
        print(f'\rEpoch [{epoch+1}/{EPOCHS}] {i}/{len(trainloader)} Loss: {running_loss:.4f} Acc : {(correct/total):.4f}', end='')
#         del data
    test_acc = get_test_acc(model,testloader)
    torch.save(model.state_dict(),'./BERT_news_2e5_' + str(epoch) + '.pkl')
    print('test acc:' , test_acc)


Some weights of the model checkpoint at ./bert_pretrain_news/ were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not init

Epoch [1/5] 6915/18870 Loss: 6301.8326 Acc : 0.5115